Zhang Chao, 2025/04/03<br>

In [1]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import geopandas as gpd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.patches as patches
import sys
import os
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
BGC = xr.open_dataset('bgc_terraclimate_Yr_CN_1km_2001_2020.nc')
dLSTd = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020.nc')
dLSTn = xr.open_dataset('delta_LSTnight_Yr_CN_2001_2020.nc')
dEVI  = xr.open_dataset('delta_EVI_Yr_CN_2001_2020.nc')

shp_cn = gpd.read_file('../shapefile/ChinaAll.shp')
shp_nanhai = gpd.read_file('../shapefile/Nanhai.shp')
shp_climzone = gpd.read_file('../shapefile/ClimateZone_3.shp')

In [3]:
def get_vif_only(iwu, pr, ta, sr, ws, devi, region):
    df = pd.DataFrame({
        'dIwu': iwu[region].to_list(),
        'Pr': pr[region][2:].to_list(),
        'Ta': ta[region][2:].to_list(),
        'Sr': sr[region][2:].to_list(),
        'Ws': ws[region][2:].to_list(),
        'dEvi': devi[region][2:].to_list(),
    })

    # Add constant for intercept
    x = sm.add_constant(df)
    
    # Compute VIF
    vif_data = pd.DataFrame()
    vif_data["variable"] = x.columns
    vif_data["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
    
    # Return as dictionary without constant
    vif_dict = dict(zip(vif_data["variable"], vif_data["VIF"]))
    vif_dict.pop('const', None)  # remove intercept if present

    return vif_dict

In [4]:
df_dLSTday   = pd.read_csv('regionmean_dLSTday_Yr.csv')
df_dLSTnight = pd.read_csv('regionmean_dLSTnight_Yr.csv')
df_IWU       = pd.read_csv('regionmean_IWU_Yr.csv')
df_pr        = pd.read_csv('regionmean_pr.csv')
df_sr        = pd.read_csv('regionmean_sr.csv')
df_ta        = pd.read_csv('regionmean_ta.csv')
df_ws        = pd.read_csv('regionmean_ws.csv')
df_dEVI      = pd.read_csv('regionmean_dEVI.csv')

In [5]:
'''
Get the regional trend contributed by iwu, background climate, and evi
'''
regions=['China','Arid','Semi','Humid']
vifList= [get_vif_only(df_IWU,df_pr,df_ta,df_sr,df_ws,df_dEVI,region) for region in regions]
df_vif = pd.DataFrame(vifList)
df_vif.index = regions
df_vif = df_vif.T


In [ ]:
colors = ['#87CEFA','#3CB371','#C1AB96','#F6CB60','#CAB2D6','#B2DF8A']

def plot_bars(ax,mean,ylim,title,ylabel,legendFlag):
    ax.set_ylim(ylim)
    ax.bar(range(1,len(mean)+1),mean,color=colors)
    ax.set_ylabel(ylabel)
    ax.set_xticks([])
    ax.set_title(title)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # Create custom legend handles
    handles = [
        patches.Patch(color=colors[0], label='IWU'),
        patches.Patch(color=colors[1], label='Precipitation'),
        patches.Patch(color=colors[2], label='Air temperature'),
        patches.Patch(color=colors[3], label='Solar radiation'),
        patches.Patch(color=colors[4], label='Wind speed'),
        patches.Patch(color=colors[5], label='$\Delta$EVI'),
    ]
    
    # Add legend to the main plot
    if legendFlag:
        ax.legend(handles=handles, ncol=2, loc='best',frameon=False,
                  columnspacing = 3)

    return ax


fig = plt.figure(figsize=(9,5))
plot_settings()
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(9,6),sharex=False)
plot_settings()

ylim = [0,10]
ylabel='Variance Inflation factor'
titles = ['China','Arid','Semi-arid/humid','Humid']
plot_bars(axes[0,0],df_vif['China'],ylim,titles[0],ylabel,True)
plot_bars(axes[0,1],df_vif['Arid'] ,ylim,titles[1],'',False)
plot_bars(axes[1,0],df_vif['Semi'] ,ylim,titles[2],ylabel,False)
plot_bars(axes[1,1],df_vif['Humid'],ylim,titles[3],'',False)

plt.tight_layout()
plt.savefig('../figures/Figure_S19.png',dpi=300)